In [54]:
# import packages
import pandas as pd

# Load your 4 files
df1 = pd.read_csv('/storage/home/hcoda1/6/sali363/p-dsgt_clef2025-0/exist2025/test_predictions_with_probs.csv')
df2 = pd.read_csv('/storage/home/hcoda1/6/sali363/p-dsgt_clef2025-0/exist2025/test_predictions_with_probs.csv')
df3 = pd.read_csv('/storage/home/hcoda1/6/sali363/p-dsgt_clef2025-0/exist2025/test_predictions_with_probs.csv')
df4 = pd.read_csv('/storage/home/hcoda1/6/sali363/p-dsgt_clef2025-0/exist2025/test_predictions_with_probs.csv')

In [55]:
df1.head()

,text,path_video,target,predicted_prob
0,i’m getting so use to this now unamused_face ...,videos/6910747250536893701.mp4,1,0.357164
1,"reply to to perform the stretch, fall forward ...",videos/6998726239087250693.mp4,0,0.410588
2,"it's the small things, p.s these are only joke...",videos/6898724802001997057.mp4,1,0.360207
3,"women we are so funny. j, women, women, wome...",videos/6977700217835638021.mp4,1,0.476894
4,"we all waited. alright y'all, this is the ne...",videos/6841534512518761733.mp4,0,0.370917


In [56]:
# Merge them on the 'id' column
merged = df1.merge(df2, on='path_video', suffixes=('', '_df2'))
merged = merged.merge(df3, on='path_video', suffixes=('', '_df3'))
merged = merged.merge(df4, on='path_video', suffixes=('', '_df4'))

df = merged

In [57]:
cols_to_drop = ['text','text_df2','text_df3','text_df4'
                ,'target_df3','target_df4','path_video']
X = X.drop(columns=cols_to_drop, errors='ignore')

X.head()


,predicted_prob,predicted_prob_df2,predicted_prob_df3,predicted_prob_df4
0,0.357164,0.357164,0.357164,0.357164
1,0.410588,0.410588,0.410588,0.410588
2,0.360207,0.360207,0.360207,0.360207
3,0.476894,0.476894,0.476894,0.476894
4,0.370917,0.370917,0.370917,0.370917


In [58]:
# Separate features and target
X = X.drop(columns=['target'])
y = df['target']

KeyError: "['target'] not found in axis"

In [ ]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

class TabularDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X.values, dtype=torch.float32)
        self.y = torch.tensor(y.values, dtype=torch.float32).unsqueeze(1)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]


In [ ]:
import torch.nn as nn

class MLPEnsemble(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1),
            nn.Sigmoid()  # For binary classification
        )

    def forward(self, x):
        return self.net(x)


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

# Initial 80/20 split for temp and test
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Now split temp into train/val (75/25 of 80 = 60/20)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=42)


In [ ]:
X_train

In [ ]:
train_ds = TabularDataset(X_train, y_train)
val_ds = TabularDataset(X_val, y_val)
test_ds = TabularDataset(X_test, y_test)

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=32)
test_loader = DataLoader(test_ds, batch_size=32)


In [59]:
model = MLPEnsemble(input_dim=X.shape[1])
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(10):
    model.train()
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
    
    model.eval()
    with torch.no_grad():
        val_losses = [criterion(model(Xb), yb) for Xb, yb in val_loader]
        val_loss = torch.stack(val_losses).mean()
    print(f"Epoch {epoch+1}, Validation Loss: {val_loss:.4f}")


Epoch 1, Validation Loss: 0.6960
Epoch 2, Validation Loss: 0.6976
Epoch 3, Validation Loss: 0.7012
Epoch 4, Validation Loss: 0.7052
Epoch 5, Validation Loss: 0.7027
Epoch 6, Validation Loss: 0.7002
Epoch 7, Validation Loss: 0.6988
Epoch 8, Validation Loss: 0.6955
Epoch 9, Validation Loss: 0.6923
Epoch 10, Validation Loss: 0.6889


In [60]:
def evaluate(loader, y_true):
    model.eval()
    preds = []
    with torch.no_grad():
        for xb, _ in loader:
            pred = model(xb)
            preds.append(pred)
    preds = torch.cat(preds).squeeze().numpy()
    preds_bin = (preds > 0.5).astype(int)
    return accuracy_score(y_true, preds_bin), f1_score(y_true, preds_bin)


In [61]:
train_acc, train_f1 = evaluate(train_loader, y_train)
val_acc, val_f1 = evaluate(val_loader, y_val)
test_acc, test_f1 = evaluate(test_loader, y_test)

print("\n🏋️ Final Scores:")
print(f"Train     — Acc: {train_acc:.4f}, F1: {train_f1:.4f}")
print(f"Validation— Acc: {val_acc:.4f}, F1: {val_f1:.4f}")
print(f"Test      — Acc: {test_acc:.4f}, F1: {test_f1:.4f}")



🏋️ Final Scores:
Train     — Acc: 0.5214, F1: 0.6782
Validation— Acc: 0.5641, F1: 0.6792
Test      — Acc: 0.5128, F1: 0.6545
